In [14]:
# from libs.chatglm3 import ChatGLM3
# MODEL_PATH = "D:\Github\Repositories\models\huggingface\chatglm3-6b"
# llm = ChatGLM3()
# llm.load_model(MODEL_PATH)

In [ ]:
from libs.qwen18 import Qwen
MODEL_PATH = "d:\Github\modelscope\qwen\Qwen-1_8B-Chat-Int4"
llm = Qwen()
llm.load_model(MODEL_PATH)

In [18]:
import torch
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
from langchain_community.llms import Tongyi

In [19]:

# llm = Tongyi()

In [20]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredFileLoader
loader = DirectoryLoader('./static/books/', glob="**/*.txt")
# loader = UnstructuredFileLoader('./static/books/2022版中国乳腺癌随诊随访与健康管理指南.txt')
# 中国抗癌协会乳腺癌诊治指南与规范（2024年版）.txt
# 2022版中国乳腺癌随诊随访与健康管理指南.txt
docs = loader.load()

In [23]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-small-zh-v1.5')

In [24]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=32)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [25]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [26]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [33]:
torch.cuda.empty_cache()
response = retrieval_chain.invoke({"input": "曲妥珠单抗诱导的心肌病表现是什么?"})
print(response["answer"])

曲妥珠单抗诱导的心肌病主要表现为治疗期间左心室射血分数（LVEF）无症状性的下降，而有症状的显性心力衰竭并不常见。当患者出现心力衰竭症状且LVEF较基线值下降超过5%且其绝对值低于55%，或者无心力衰竭症状的患者LVEF较基线值下降超过10%且绝对值同样低于55%时，可以判断为心脏不良反应。此外，曲妥珠单抗引起的心脏功能不全似乎不具备剂量依赖性，并且在停止治疗后，心脏不良反应通常可以逆转。


In [14]:
llm.history=[]